# Estimation on simulated data using MSM

In [1]:
from consav import runtools
runtools.write_numba_config(disable=0,threads=4)

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Local modules
from Model import RetirementClass
import figs
import SimulatedMinimumDistance as SMD

# Global modules
import numpy as np

# Singles

In [ ]:
def weight_matrix(std):
    std_inv = np.zeros(std.shape)
    idx = np.nonzero(std>0)[0]
    std_inv[idx] = 1/std[idx]
    return np.eye(std.size)*std_inv

Single = RetirementClass(simN=1e5,simT=12)
Single.solve()
Single.simulate()
mom_data = SMD.MomFunSingle(Single.sim,Single.par)
mom_se = SMD.MomFunSingle(Single.sim,Single.par,std=True)
weight = weight_matrix(mom_se)

Set up model for estimation

In [ ]:
Single.sim_seed = Single.sim_seed + 1000 # set new seed for estimation
Single.recompute()

In [ ]:
# function to generate moments
def mom_fun(sim):
    return SMD.MomFunSingle(sim,Single.par)

Set up estimation

In [8]:
# Parameters to be estimated (and guess)
true = [Single.par.alpha_0_male, Single.par.alpha_0_female, Single.par.alpha_1, Single.par.sigma_eta]
theta0 = true
add_str = '_est'
est_par = ["alpha_0_male", "alpha_0_female", "alpha_1", "sigma_eta"]

# Setup estimation
smd = SMD.SimulatedMinimumDistance(Single,mom_data,mom_fun,print_iter=[True,100],options={'disp':True})
smd.est_par = est_par

Run estimation

In [9]:
tic = time.time()
smd_base.estimate(theta0,weight)
theta = smd.est
toc = time.time()

Iteration: 50 (0.73 minutes)
 alpha_0_male=0.1629 alpha_0_female=0.2254 alpha_1=0.0701 sigma_eta=0.5243 -> 49.6945
Iteration: 100 (0.72 minutes)
 alpha_0_male=0.1463 alpha_0_female=0.1490 alpha_1=0.0700 sigma_eta=0.4121 -> 3.0982
Iteration: 150 (0.74 minutes)
 alpha_0_male=0.1481 alpha_0_female=0.1496 alpha_1=0.0698 sigma_eta=0.4166 -> 3.0131
Iteration: 200 (0.73 minutes)
 alpha_0_male=0.1480 alpha_0_female=0.1496 alpha_1=0.0698 sigma_eta=0.4168 -> 3.0012
Iteration: 250 (0.75 minutes)
 alpha_0_male=0.1480 alpha_0_female=0.1495 alpha_1=0.0698 sigma_eta=0.4167 -> 2.9942
Optimization terminated successfully.
         Current function value: 2.992020
         Iterations: 149
         Function evaluations: 268


Result

In [10]:
print('Parameter:       ', est_par)
print('True:            ', np.round(true, 4))
print('Estimated:       ', np.round(theta, 4))
print('bias:            ', np.round(theta-true, 4))
print('Estimation time: ', round((toc-tic)/60,1), 'minutes')

Parameter:        ['alpha_0_male', 'alpha_0_female', 'alpha_1', 'sigma_eta']
True:             [0.16  0.16  0.053 0.435]
Estimated:        [0.148 0.15  0.07  0.417]
bias:             [-0.012 -0.01   0.017 -0.018]
Estimation time:  3.93 minutes


clean up

In [ ]:
%reset -f

# Couples

In [ ]:
from consav import runtools
runtools.write_numba_config(disable=0,threads=4)

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Local modules
from Model import RetirementClass
import figs
import SimulatedMinimumDistance as SMD

# Global modules
import numpy as np

In [ ]:
# create weight matrix
def weight_matrix(std):
    std_inv = np.zeros(std.shape)
    idx = np.nonzero(std>0.01)[0]
    std_inv[idx] = 1/std[idx]
    return np.eye(std.size)*std_inv

Couple = RetirementClass(couple=True, simN=1e6, simT=12)
Couple.solve()    
Couple.simulate()
mom_data = SMD.MomFunCouple(Couple.sim,Couple.par)
mom_se = SMD.MomFunCouple(Couple.sim,Couple.par,std=True)
weight = weight_matrix(mom_se)

Set up model for estimation 

In [ ]:
Couple.sim_seed = Couple.sim_seed + 1000 # set new seed for estimation
Couple.recompute()

In [ ]:
# function to generate moments
def mom_fun(sim):
    return SMD.MomFunSingle(sim,Couple.par)

Set up estimation

In [ ]:
# Parameters to be estimated (and guess)
true = [Couple.par.alpha_0_male, Couple.par.alpha_0_female, Couple.par.alpha_1, Couple.par.sigma_eta, 
        Couple.par.phi_0_male, Couple.par.phi_0_female, Couple.par.phi_1]
theta0 = true
add_str = '_est'
est_par = ["alpha_0_male", "alpha_0_female", "alpha_1", "sigma_eta", "phi_0_male", "phi_0_female", "phi_1"]

# Setup estimation
smd = SMD.SimulatedMinimumDistance(Couple,mom_data,mom_fun,print_iter=[True,50],options={'disp':True})
smd.est_par = est_par

Run estimation

In [ ]:
tic = time.time()
smd_base.estimate(theta0,weight)
theta = smd.est
toc = time.time()

Results

In [ ]:
print('Parameter:       ', est_par)
print('True:            ', np.round(true, 4))
print('Estimated:       ', np.round(theta, 4))
print('bias:            ', np.round(theta-true, 4))
print('Estimation time: ', round((toc-tic)/(60**2),1), ' hours')